## Want to Publish Stock Alerts to Slack?

Before getting started please confirm the [the notebook-integration docker stack is running](https://github.com/AlgoTraders/stock-analysis-engine#jupyter).

### Publishing FinViz Screeners to Slack

Please edit your ```./compose/envs/.env``` file and add the following Slack webhook to your docker environment variables:
    
```
SLACK_WEBHOOK=https://hooks.slack.com/services/<your remaining webhook path>
```

### Restart the Stack

Please restart the stack so the new ``SLACK_WEBHOOK`` environment variable can take effect. If you are using docker compose please restart the stack with the commands:

```
$ ./compose/stop.sh -j
$ ./compose/start.sh -j
```

### Pick a FinViz Screener

This notebook uses [a simple NYSE Dow Jones Index Financials with a P/E above 5 screener url](https://finviz.com/screener.ashx?v=111&f=exch_nyse,fa_pe_o5,idx_dji,sec_financial&ft=4) to fetch a list tickers and a ``pandas.DataFrame`` from the resulting html table.

In [1]:
# After you build a FinViz screen just paste the url here:
url = 'https://finviz.com/screener.ashx?v=111&f=exch_nyse,fa_pe_o5,idx_dji,sec_financial&ft=4'

### Imports

In [2]:
import datetime
import pandas as pd
from IPython.display import display
from IPython.display import HTML
from analysis_engine.consts import SUCCESS
from analysis_engine.consts import SLACK_FINVIZ_COLUMNS
from analysis_engine.consts import get_status
from analysis_engine.consts import ev
from analysis_engine.finviz.fetch_api import fetch_tickers_from_screener
from analysis_engine.send_to_slack import post_df
from analysis_engine.send_to_slack import post_cb

### Fetch Tickers from FinViz Screener

In [3]:
print('Fetching tickers from url: {}\n'.format(url))
res = fetch_tickers_from_screener(url=url)

df = None

if res['status'] != SUCCESS:
    print('Failed to fetch tickers from FinViz url={} with error={} and status={}'.format(url, res['err'], get_status(status=res['status'])))
else:
    print('Found tickers: {}'.format(res['rec']['tickers']))
    df = res['rec']['data']
    display(df)

Fetching tickers from url: https://finviz.com/screener.ashx?v=111&f=exch_nyse,fa_pe_o5,idx_dji,sec_financial&ft=4

2018-10-17 23:51:08,371 - analysis_engine.finviz.fetch_api - INFO - fz-screen-converter fetching url=https://finviz.com/screener.ashx?v=111&f=exch_nyse,fa_pe_o5,idx_dji,sec_financial&ft=4
2018-10-17 23:51:09,963 - analysis_engine.finviz.fetch_api - INFO - fz-screen-converter fetch done - df=5 from url=https://finviz.com/screener.ashx?v=111&f=exch_nyse,fa_pe_o5,idx_dji,sec_financial&ft=4 with tickers=['AXP', 'GS', 'JPM', 'TRV', 'V'] rows=5
Found tickers: ['AXP', 'GS', 'JPM', 'TRV', 'V']


,change,company,country,industry,market_cap,pe,price,sector,ticker,ticker_id,volume
0,0.07%,American Express Company,USA,Credit Services,89.73B,15.30,104.34,Financial,AXP,1,"3,789,094"
1,2.97%,"The Goldman Sachs Group, Inc.",USA,Investment Brokerage - National,85.83B,9.61,228.28,Financial,GS,2,"4,951,867"
2,1.11%,JPMorgan Chase & Co.,USA,Money Center Banks,373.20B,13.67,109.83,Financial,JPM,3,"18,650,929"
3,1.35%,"The Travelers Companies, Inc.",USA,Property & Casualty Insurance,34.03B,16.11,126.44,Financial,TRV,4,"2,106,779"
4,0.50%,Visa Inc.,USA,Credit Services,302.64B,39.29,142.45,Financial,V,5,"9,167,145"


### Publish the FinViz HTML Table to Slack

With your ``SLACK_WEBHOOK`` environment variable in place, Jupyter should be able to publish the FinViz's screener table to Slack.

In [4]:
if ev('SLACK_WEBHOOK', False):
    # Post the pandas.DataFrame as a Jupyter code block message to Slack
    post_df(
        df=df[SLACK_FINVIZ_COLUMNS],
        columns=SLACK_FINVIZ_COLUMNS)
else:
    print('Missing the SLACK_WEBHOOK environment variable\nUnable to publish to Slack without this environment variable')

2018-10-17 23:51:10,312 - analysis_engine.send_to_slack - INFO - Successful post of attachment=True to slack_webhook
2018-10-17 23:51:10,551 - analysis_engine.send_to_slack - INFO - Successful post of attachment=True to slack_webhook
